In [54]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

# Elos mensuels
Nous allons récupérer toutes les elos mensuels pour chaque joueur titrés

In [297]:
df2 = pd.read_csv('titled_players.csv', index_col = 0)

Récupérons le classement elos uniquement lorsqu'il y'a au moins 1 valeur pour une des trois catégorie classique, rapide ou blitz

In [298]:
df2[['classique', 'rapide', 'blitz']].isnull().sum(axis=1).value_counts()

0    10273
2     5831
1     3335
3       70
dtype: int64

In [299]:
id_any_rating = df2[df2[['classique', 'rapide', 'blitz']].isnull().sum(axis=1) <= 2]['id']
id_any_rating.head()

1     4611870
2     2002310
3    14100657
4     4101529
5     4103424
Name: id, dtype: int64

In [150]:
# %%time
## Le code risque de prendre environ 2h, enregistrons le directement en un csv
def scrap_rating():
    """Scrap des données mensuel de tous les joueurs titrés sur le site de la FIDE
    
        Returns: 
            DataFrame : L'id du joueur, nombre de parties par catégorie, ainsi que le classement 
            (categorie : classique, rapide ou blitz)
    """
    try:
        df_merged = pd.DataFrame()
        for id_fide in id_any_rating:
            url = 'https://ratings.fide.com/profile/'+str(id_fide)+'/chart'
            html_text = requests.get(url).text
            soup = BeautifulSoup(html_text, 'html.parser')
            
            soup.find_all('table', class_="profile-table profile-table_chart-table")
            rating_text = []

            for td in soup.find_all('table', class_="profile-table profile-table_chart-table"):
                soup_td = td.find_all('td')

            for i in soup_td:
                rating_text.append(re.sub(r'(\xa0)+',' ', i.text))
            if len(rating_text)% 7 != 0:
                # cas où l'extraction des données se passe mal
                continue
            array = np.array(rating_text).reshape((-1,7))
            
            cols = ['id',
                'date', 
                'classique_elo', 
                'classique_parties', 
                'rapide_elo', 
                'rapide_parties', 
                'blitz_elo', 
                'blitz_parties'
                ]
            df = pd.DataFrame(data=array,columns=cols[1:])
            df['id'] = id_fide
            df = df[cols]
            df_merged = pd.concat([df_merged,df])

        return df_merged.reset_index(drop=True)
    except ValueError:
        print(id_fide)
        return df_merged.reset_index(drop=True), id_fide
df = scrap_rating()
df.to_csv('players_rating.csv')

In [334]:
# df = pd.read_csv('players_rating.csv', index_col = 0)

In [335]:
df.head()

,id,date,classique_elo,classique_parties,rapide_elo,rapide_parties,blitz_elo,blitz_parties
0,4611870,2021-Sep,2290,0,,,,
1,4611870,2021-Aug,2290,0,,,,
2,4611870,2021-Jul,2290,0,,,,
3,4611870,2021-Jun,2290,0,,,,
4,4611870,2021-May,2290,0,,,,


Il y'a 6 joueurs dont on n'a pas pu récupérer les données 

In [336]:
len(id_any_rating) - len(df['id'].unique())

6

In [337]:
joueurs = np.setdiff1d(id_any_rating.unique(), df['id'].unique())
joueurs

array([  701327,   701599,  4106970,  4150813,  4633563, 14105152],
      dtype=int64)

In [338]:
df2[df2['id'].isin(joueurs)]

,id,nom,titre,woman_titre,pays,classique,rapide,blitz,naissance,sexe,actif,titre_dates,Woman Candidate Master (WCM),Candidate Master (CM),Woman FIDE Master (WFM),FIDE Master (FM),Woman International Master (WIM),International Master (IM),Woman Grandmaster (WGM),Grandmaster (GM)
679,701599,"Demeter, Istvan",FM,NaN,HUN,1995.0,NaN,NaN,1946.0,M,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2106,14105152,"Chemeris, Roman",FM,NaN,UKR,2333.0,NaN,NaN,1956.0,M,0,"['FIDE Master (FM)', '2007']",NaN,NaN,NaN,2007.0,NaN,NaN,NaN,NaN
5054,4106970,"Stepovaia, Tatiana",WGM,WGM,RUS,2305.0,NaN,NaN,1965.0,F,0,"['Woman Grandmaster (WGM)', '1992', 'Woman Int...",NaN,NaN,NaN,NaN,1988.0,NaN,1992.0,NaN
7984,701327,"Fancsy, Imre",IM,NaN,HUN,2346.0,2371.0,2349.0,1974.0,M,1,"['International Master (IM)', '1993']",NaN,NaN,NaN,NaN,NaN,1993.0,NaN,NaN
8259,4150813,"Apalev, Sergey",FM,NaN,RUS,2149.0,2098.0,NaN,1975.0,M,0,"['FIDE Master (FM)', '2006']",NaN,NaN,NaN,2006.0,NaN,NaN,NaN,NaN
8951,4633563,"Eichler, Christoph",FM,NaN,GER,2301.0,NaN,NaN,1977.0,M,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Nettoie un peu les données

In [339]:
df = df.replace({' ': np.nan}).copy()

In [340]:
for i in df.columns[1:]:
    df[i] = df[i].str.strip()

In [356]:
# Convertir les colonnes de types objets en float
numeric_cols = df.head().select_dtypes('object').columns

In [359]:
df[numeric_cols] = df[numeric_cols].astype('float')

In [363]:
from datetime import datetime
date_parser = lambda x: datetime.strptime(x,'%Y-%b')

In [364]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%b')

Les fichiers étants volimineux nous allons les ziper pour les rendre plus léger

In [365]:
df.to_csv('players_rating_clean.csv')
df.to_pickle('players_rating_clean.zip')

In [278]:
# df = pd.read_csv('players_rating_clean.csv', index_col = 0, date_parser = date_parser, parse_dates = [2])

Vérifions que le zip marche bien

In [367]:
data = pd.read_pickle('players_rating_clean.zip')
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2835829 entries, 0 to 2835828
Data columns (total 8 columns):
 #   Column             Dtype         
---  ------             -----         
 0   id                 int64         
 1   date               datetime64[ns]
 2   classique_elo      float64       
 3   classique_parties  float64       
 4   rapide_elo         float64       
 5   rapide_parties     float64       
 6   blitz_elo          float64       
 7   blitz_parties      float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 194.7 MB
None


,id,date,classique_elo,classique_parties,rapide_elo,rapide_parties,blitz_elo,blitz_parties
0,4611870,2021-09-01,2290.0,0.0,NaN,NaN,NaN,NaN
1,4611870,2021-08-01,2290.0,0.0,NaN,NaN,NaN,NaN
2,4611870,2021-07-01,2290.0,0.0,NaN,NaN,NaN,NaN
3,4611870,2021-06-01,2290.0,0.0,NaN,NaN,NaN,NaN
4,4611870,2021-05-01,2290.0,0.0,NaN,NaN,NaN,NaN


In [366]:
# pd.read_csv('players_rating_clean.csv', index_col = 0).info()